In [1]:
from csb_multi_env import *

2024-05-12 23:22:05,006	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-12 23:22:05,870	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-12 23:22:06,119	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
env = CodersStrikeBackMulti(dt=1)

In [3]:
env.reset()

({'car_0_0': array([ 3.00000000e+00, -2.31993829e+00,  7.14934290e+03,  6.47086792e+03,
          0.00000000e+00,  0.00000000e+00,  2.59900000e+03,  1.57800000e+03,
          4.40400000e+03,  3.45600000e+03,  1.13920000e+04,  7.85300000e+03,
          9.63300000e+03,  5.44500000e+03,  8.19500000e+03,  3.65900000e+03]),
  'car_1_0': array([ 3.00000000e+00, -2.49274611e+00,  7.56760574e+03,  5.34612075e+03,
          0.00000000e+00,  0.00000000e+00,  2.59900000e+03,  1.57800000e+03,
          4.40400000e+03,  3.45600000e+03,  1.13920000e+04,  7.85300000e+03,
          9.63300000e+03,  5.44500000e+03,  8.19500000e+03,  3.65900000e+03]),
  'car_2_0': array([ 3.00000000e+00, -2.68540712e+00,  7.98586858e+03,  4.22137358e+03,
          0.00000000e+00,  0.00000000e+00,  2.59900000e+03,  1.57800000e+03,
          4.40400000e+03,  3.45600000e+03,  1.13920000e+04,  7.85300000e+03,
          9.63300000e+03,  5.44500000e+03,  8.19500000e+03,  3.65900000e+03]),
  'car_0_1': array([ 3.00000000e+00, 

In [4]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    done = done['__all__']
    rewards.append(reward)

In [5]:
rewards[-1]

{'car_0_0': 1,
 'car_1_0': -1,
 'car_2_0': -1,
 'car_0_1': 1,
 'car_1_1': -1,
 'car_2_1': -1}

In [6]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [7]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

2024-05-12 23:22:31,543	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.14
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8265


In [51]:
agent_ids = CodersStrikeBackMulti()._agent_ids
sym_policies = {agent_id: f"policy_{agent_ids[0]}" for agent_id in agent_ids}  # Symmetric
asym_policies = {agent_id: f"policy_{agent_id}" for agent_id in agent_ids}  # Asymmetric

def policy_mapping_fn(agent_id, episode, worker, *, policies=None, **kwargs):
    return policies[agent_id]


policies = sym_policies
policy_mapping = partial(policy_mapping_fn, policies=policies)

config = (
        PPOConfig()
        .environment(CodersStrikeBackMulti)
        .framework('torch')
        .training(train_batch_size=1024, gamma=0.99)
        .resources(num_gpus=0)
        .rollouts(num_rollout_workers=3)
        .multi_agent(policies=list(policies.values()),
                     policy_mapping_fn=policy_mapping)
    )

stop = {"timesteps_total": 12000}

# THE WAY OF CODING IS A BIT DIFFERENT HERE FROM THE SINGLE AGENT ONE.
tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(stop=stop),
    )

res = tuner.fit()
best_result = res.get_best_result(metric="loss", mode="min")
best_checkpoint = best_result.checkpoint


2024-05-12 22:51:55,185	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-12 22:52:42,223	INFO tune.py:1143 -- Total run time: 47.05 seconds (46.58 seconds for the tuning loop).


# Play against the above trained strategy

In [8]:
import random 

class CSBSelfPlay(MultiAgentEnv):
    def __init__(self, seed=None):
        # These are all the possible agents (main can play as team 0 and 1.  Old can only play team 1)
        self.env = CodersStrikeBackMulti(dt=1)
        self._agent_ids = []
        for team in self.env.teams:
            for car in range(self.env.num_racers_per_team):
                self._agent_ids.append(f'main_{team}_{car}')
                if team >0:
                    self._agent_ids.append(f'old_{team}_{car}')
                    self._agent_ids.append(f'exploiter_{team}_{car}')
        # Override default action space so that it covers all possible strategies
        self.observation_space = gym.spaces.Dict(make_agent_dictionary(self._agent_ids, self.env.ind_observation_space))
        self.action_space = gym.spaces.Dict(make_agent_dictionary(self._agent_ids, self.env.ind_action_space))
    
    def reset(self, **kwargs):
        self.create_competitor_pool()
        return self.env.reset(**kwargs)

    def create_competitor_pool(self):
        selected_ids = []
        policies = ['main'] + random.choices(['main','old','exploiter'], weights=[0.6,0.2,0.2], k=self.env.num_teams-1)
        for team in range(self.env.num_teams):
            for car in range(self.env.num_racers_per_team):
                selected_ids.append(f'{policies[team]}_{team}_{car}')
        self.env._agent_ids = selected_ids
        for i, racer in enumerate(self.env.racers):
            racer.aid = selected_ids[i]

    def step(self, **kwargs):
        return self.env.step(**kwargs)
        # observations, rewards, dones, trucates, infos = self.env.step(actions)
        # return observations, self.modify_rewards(rewards), dones, trucates, infos 

    def competitive_results(self, agent_id, reward, max_reward):
        if agent_id.startswith('exploiter'): # exploiters get huge payouts for beating the main agent
            return reward*5 if reward == max_reward else -reward*5
        return reward

    def modify_rewards(self, rewards):
        max_reward = max(rewards.values())
        competitive_rewards = {aid: self.competitive_results(aid, r, max_reward) for aid, r in rewards.items()}
        return competitive_rewards

In [9]:
env = CSBSelfPlay()

In [10]:
env.reset()
env._agent_ids

['main_0_0',
 'main_0_1',
 'main_1_0',
 'old_1_0',
 'exploiter_1_0',
 'main_1_1',
 'old_1_1',
 'exploiter_1_1',
 'main_2_0',
 'old_2_0',
 'exploiter_2_0',
 'main_2_1',
 'old_2_1',
 'exploiter_2_1']

In [11]:
env.reset()
print(env.env._agent_ids)
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(actions=env.action_space.sample())
    done = done['__all__']
    rewards.append(reward)

['main_0_0', 'main_0_1', 'exploiter_1_0', 'exploiter_1_1', 'main_2_0', 'main_2_1']


In [12]:
reward

{'main_0_0': 1,
 'main_0_1': -1,
 'exploiter_1_0': 5,
 'exploiter_1_1': 5,
 'main_2_0': -1,
 'main_2_1': -1}

In [13]:
agent_ids = CSBSelfPlay()._agent_ids
sym_policies = {agent_id: f"policy_{agent_ids[0]}" for agent_id in agent_ids}  # Symmetric
asym_policies = {agent_id: f"policy_{agent_id}" for agent_id in agent_ids}  # Asymmetric

def policy_mapping_fn(agent_id, episode, worker, *, policies=None, **kwargs):
    return policies[agent_id]


policies = asym_policies
policy_mapping = partial(policy_mapping_fn, policies=policies)


In [16]:
config = (
        PPOConfig()
        .environment(CSBSelfPlay)
        .framework('torch')
        .training(train_batch_size=1024, gamma=0.99)
        .resources(num_gpus=0)
        .rollouts(num_rollout_workers=3)
        .multi_agent(policies=list(policies.values()),
                     policy_mapping_fn=policy_mapping)
    )

stop = {"timesteps_total": 12000}

# THE WAY OF CODING IS A BIT DIFFERENT HERE FROM THE SINGLE AGENT ONE.
tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(stop=stop),
    )

res = tuner.fit()
best_result = res.get_best_result(metric="loss", mode="min")
best_checkpoint = best_result.checkpoint

2024-05-12 23:24:14,232	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-12 23:24:18,693	ERROR tune_controller.py:1502 -- Trial task failed for trial PPO_CSBSelfPlay_6b213_00000
Traceback (most recent call last):
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", li